In [1]:
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, Audio
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    TrainingArguments,
    Trainer
)
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import re
import json
from tqdm import tqdm
import evaluate


# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configuration
model_name = "facebook/wav2vec2-base" # Pre-trained model
output_dir = "./wav2vec2-finetuned-timit"
train_csv = "train.csv"
test_csv = "test.csv"
max_duration_in_seconds = 20.0  # Maximum audio clip duration
sample_rate = 16000  # Standard for wav2vec2
batch_size = 8  # Adjust based on GPU memory
num_train_epochs = 3
learning_rate = 3e-4
tokenizer_max_length = 30

C:\Users\arsha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [3]:
from datasets import load_dataset, Audio

dataset = load_dataset(
    "csv",
    data_files={"train": train_csv, "test": test_csv},
    delimiter=","
)

# Cast the "path" column to Audio
dataset = dataset.cast_column("path", Audio(sampling_rate=16_000))
# If this works then this should be some numbers.
print(dataset["train"][694])


{'path': {'path': 'TIMIT/data/TRAIN\\DR2\\MDMT0\\VOCODED_SI572.WAV.wav', 'array': array([-3.05175781e-05,  1.22070312e-04,  2.13623047e-04, ...,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), 'sampling_rate': 16000}, 'transcription': '"SEWING BRINGS NUMBNESS" WRITES: WHAT MAKES MY HANDS NUMB WHEN SEWING?'}


In [4]:
 
def preprocess(batch):
    batch["input_values"] = batch["path"]["array"]
    batch["input_length"] = len(batch["input_values"])
    return batch

dataset = dataset.map(preprocess, remove_columns=["path"])

In [12]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base")

Downloading:   4%|▎         | 42.2M/1.18G [00:15<00:59, 20.5MB/s]C:\Users\arsha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
C:\Users\arsha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytor

In [14]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["input_values"], sampling_rate=16_000).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=["transcription", "input_length"])


Map:   0%|          | 0/4620 [00:00<?, ? examples/s]


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).